<a href="https://colab.research.google.com/github/cmd1301/ny_airbnb_PYTHON/blob/main/ny_airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import io
import numpy as np
import plotly.express as px
import folium

In [2]:
from google.colab import files

In [3]:
up = files.upload()

Saving AB_NYC_2019.csv to AB_NYC_2019.csv


In [4]:
df = pd.read_csv(io.BytesIO(up['AB_NYC_2019.csv']))
pd.read_csv(io.BytesIO(up['AB_NYC_2019.csv'])).head()

id                                              name  host_id  \
0  2539                Clean & quiet apt home by the park     2787   
1  2595                             Skylit Midtown Castle     2845   
2  3647               THE VILLAGE OF HARLEM....NEW YORK !     4632   
3  3831                   Cozy Entire Floor of Brownstone     4869   
4  5022  Entire Apt: Spacious Studio/Loft by central park     7192   

     host_name neighbourhood_group neighbourhood  latitude  longitude  \
0         John            Brooklyn    Kensington  40.64749  -73.97237   
1     Jennifer           Manhattan       Midtown  40.75362  -73.98377   
2    Elisabeth           Manhattan        Harlem  40.80902  -73.94190   
3  LisaRoxanne            Brooklyn  Clinton Hill  40.68514  -73.95976   
4        Laura           Manhattan   East Harlem  40.79851  -73.94399   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0     Private room    149               1                  9  2018-10-19   
1  Entire home/apt    225               1                 45  2019-05-21   
2     Private room    150               3                  0         NaN   
3  Entire home/apt     89               1                270  2019-07-05   
4  Entire home/apt     80              10                  9  2018-11-19   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               0.21                               6               365  
1               0.38                               2               355  
2                NaN                               1               365  
3               4.64                               1               194  
4               0.10                               1                 0

In [5]:
# Seleção de colunas
# df.loc[lnhas,colunas]
df1 = df.dropna()

In [ ]:
# 1. Valor médio aluguel na cidade de NY
np.average(df.loc[:, 'price']).round(2)

152.72

In [ ]:
# 2. Nomes das regiões existentes na cidade de NY
np.unique(df.loc[:,'neighbourhood_group'])

array(['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'],
      dtype=object)

In [ ]:
# 3. Valor do aluguel diário mais caro da cidade de NY
np.max(df.loc[:, 'price'])

10000

In [ ]:
# 4. Quais são as categorias de imóveis que estão cadastradas dentro da 
# base de dados da cidade de NY?
np.unique(df.loc[:, 'room_type'])

array(['Entire home/apt', 'Private room', 'Shared room'], dtype=object)

In [ ]:
# 5. Existem quantos usuários (Hosts) únicos cadastrados dentro da base de
# dados da cidade de NY?
len(np.unique(df1.loc[:, 'host_id']))

30232

In [ ]:
# 6. Como é a variação dos preços dos imóveis em NY?
np.std(df1.loc[:, 'price']).round(2)

196.99

In [ ]:
# 7. Existem mais imóveis baratos ou caros?
px.histogram(df1, 'price', nbins=300)

In [ ]:
# 8. Qual a distribuição do número de Reviews? Existem imóveis com muitos e
# outro com poucos reviews?
px.histogram(df1, 'number_of_reviews', nbins=300)

In [13]:
# 9. Qual é o valor do aluguel (diária) mais caro de cada região da base de
# dados da cidade de NY?
col = ['neighbourhood_group', 'price']
data_plot = df1.loc[df1['availability_365'] > 0, col].groupby('neighbourhood_group').max().reset_index()
px.bar(
    data_plot,
    x = 'neighbourhood_group',
    y = 'price'
)

In [23]:
# 10. Conseguimos saber onde estão localizados os imóveis com o valor do
# aluguel mais caro, na cidade de NY?
col = ['price', 'neighbourhood_group', 'latitude', 'longitude']
data_plot = df1.loc[df1['availability_365'] > 0, col].groupby('neighbourhood_group').max().reset_index()

map = folium.Map(
    location=[data_plot['latitude'].mean(),
              data_plot['longitude'].mean()],
              zoom_start=14,
              control_scale=True
)

for index, location_info in data_plot.iterrows():
  folium.Marker([location_info['latitude'], location_info['longitude']],
                popup=location_info['neighbourhood_group']).add_to(map)
map

In [24]:
# 11. Conseguimos saber onde estão localizados os imóveis pelo seu tipo?
col = ['room_type', 'neighbourhood_group', 'latitude', 'longitude']
data_plot = df1.loc[:, col].groupby('neighbourhood_group').sample(50)
data_plot['color'] = 'NA'

map = folium.Map(
    location=[data_plot['latitude'].mean(),
              data_plot['longitude'].mean()],
              zoom_start=4,
              control_scale=True
)

data_plot.loc[data_plot['room_type'] == 'Private room', 'color'] = 'darkgreen'
data_plot.loc[data_plot['room_type'] == 'Entire home/apt', 'color'] = 'darkred'
data_plot.loc[data_plot['room_type'] == 'Shared room', 'color'] = 'darkblue'

for index, location_info in data_plot.iterrows():
  folium.Marker([location_info['latitude'], location_info['longitude']],
                popup=location_info['neighbourhood_group'],
                icon=folium.Icon(color=location_info['color'])).add_to(map)
map

In [ ]:
# Outras questões

# 1. Qual o número total de linhas do conjunto de dados?
len(df.index)

48895

In [ ]:
# 2. Calcule o valor máximo da Quantidade de Anúncios do Host do Imóvel
np.max(df.loc[:, 'calculated_host_listings_count'])

327

In [ ]:
# 3. Quais os tipos de dados de cada coluna?
df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [ ]:
# 4. Qual o número de dados faltantes do conjunto de dados?
df.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [ ]:
# 5. Selecione as seguintes colunas:
cols = ['id', 'name', 'latitude', 'longitude', 'room_type', 'price', 'reviews_per_month']
df.loc[:,cols]
  # a. Identificador do registro na base de dados
#df.loc[:,'id']
  # b. Nome do Anúncio na plataforma do Airbnb
#df.loc[:,'name']
  # c. Ponto geográfico de latitude
#df.loc[:,'latitude']
  # d. Ponto geográfico de longitude
#df.loc[:,'longitude']
  # e. Tipo de locação (quarto, casa inteira, etc)
#df.loc[:,'room_type']
  # f. Preço do aluguel (diária)
#df.loc[:,'price']
  # g. Quantidade de Revisões Mensais
#df.loc[:,'reviews_per_month']

id                                               name  latitude  \
0          2539                 Clean & quiet apt home by the park  40.64749   
1          2595                              Skylit Midtown Castle  40.75362   
2          3647                THE VILLAGE OF HARLEM....NEW YORK !  40.80902   
3          3831                    Cozy Entire Floor of Brownstone  40.68514   
4          5022   Entire Apt: Spacious Studio/Loft by central park  40.79851   
...         ...                                                ...       ...   
48890  36484665    Charming one bedroom - newly renovated rowhouse  40.67853   
48891  36485057      Affordable room in Bushwick/East Williamsburg  40.70184   
48892  36485431            Sunny Studio at Historical Neighborhood  40.81475   
48893  36485609               43rd St. Time Square-cozy single bed  40.75751   
48894  36487245  Trendy duplex in the very heart of Hell's Kitchen  40.76404   

       longitude        room_type  price  reviews_per_month  
0      -73.97237     Private room    149               0.21  
1      -73.98377  Entire home/apt    225               0.38  
2      -73.94190     Private room    150                NaN  
3      -73.95976  Entire home/apt     89               4.64  
4      -73.94399  Entire home/apt     80               0.10  
...          ...              ...    ...                ...  
48890  -73.94995     Private room     70                NaN  
48891  -73.93317     Private room     40                NaN  
48892  -73.94867  Entire home/apt    115                NaN  
48893  -73.99112      Shared room     55                NaN  
48894  -73.98933     Private room     90                NaN  

[48895 rows x 7 columns]

In [ ]:
# 6. Calcule o valor médio das diárias mínimas para locação
np.average(df.loc[:,'minimum_nights']).round(2)

7.03

In [ ]:
# 7. Calcule o valor máximo das quantidades de avaliações
np.max(df.loc[:,'number_of_reviews'])

629

In [ ]:
# 8. Quais os nomes das colunas?
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [ ]:
# 9. Calcule o valor mínimo das Quantidades de Anúncios do Host do imóvel
np.min(df.loc[:,'calculated_host_listings_count'])

1

In [ ]:
# 10. Qual o número total de colunas do conjunto de dados?
df.shape[1]

16

In [ ]:
# 11. Calcule o valor médio das quantidades de anúncios do host
np.average(df.loc[:,'calculated_host_listings_count']).round(2)

7.14

In [ ]:
# 12. Calcule o valor mediano das quantidade de anúncios do host
np.median(df.loc[:,'calculated_host_listings_count'])

1.0

In [ ]:
# 13. Calcule o valor mínimo das quantidade de avaliações
np.min(df.loc[:,'number_of_reviews'])

0

In [ ]:
df1.head()

id                                              name  host_id  \
0  2539                Clean & quiet apt home by the park     2787   
1  2595                             Skylit Midtown Castle     2845   
3  3831                   Cozy Entire Floor of Brownstone     4869   
4  5022  Entire Apt: Spacious Studio/Loft by central park     7192   
5  5099         Large Cozy 1 BR Apartment In Midtown East     7322   

     host_name neighbourhood_group neighbourhood  latitude  longitude  \
0         John            Brooklyn    Kensington  40.64749  -73.97237   
1     Jennifer           Manhattan       Midtown  40.75362  -73.98377   
3  LisaRoxanne            Brooklyn  Clinton Hill  40.68514  -73.95976   
4        Laura           Manhattan   East Harlem  40.79851  -73.94399   
5        Chris           Manhattan   Murray Hill  40.74767  -73.97500   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0     Private room    149               1                  9  2018-10-19   
1  Entire home/apt    225               1                 45  2019-05-21   
3  Entire home/apt     89               1                270  2019-07-05   
4  Entire home/apt     80              10                  9  2018-11-19   
5  Entire home/apt    200               3                 74  2019-06-22   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               0.21                               6               365  
1               0.38                               2               355  
3               4.64                               1               194  
4               0.10                               1                 0  
5               0.59                               1               129

In [ ]:
# 14. Qual o desvio padrão do aluguel diário dos imóveis?
std_price = np.std(df1.loc[:, 'price']).round(2)
avg_price = np.average(df1.loc[:, 'price']).round(2)
print(std_price, avg_price)

196.99 142.33


In [ ]:
# 15. Qual o valor médio do tempo (dias) em que o anúncio está disponível
# na plataforma?
np.average(df1.loc[:,'availability_365']).round(2)

114.89

In [ ]:
# 16. Qual o valor médio do ponto geográfico de latitude?
np.average(df1.loc[:,'latitude'])

40.72812912547333

In [ ]:
# 17. Qual o desvio padrão do tempo (dias) em que o anúncio está disponível
# na plataforma?
np.std(df1.loc[:,'availability_365']).round(2)

129.53

In [ ]:
# 18. Qual o desvio padrão das diárias mínimas para locação?
np.std(df1.loc[:,'minimum_nights']).round(2)

17.39

In [ ]:
# 19. Desenhe um histograma das diárias mínimas para locação.
px.histogram(df1.loc[df1['minimum_nights'] < 100, :], 'minimum_nights', nbins=200)

In [ ]:
# 20. Qual a data mais antiga da última avaliação feita?
np.min(df1.loc[:,'last_review'])

'2011-03-28'

In [ ]:
# 21. Qual o menor valor do tempo (dias) em que o anúncio está disponível
# na plataforma?
np.min(df1.loc[:,'availability_365'])

0

In [ ]:
# 22. Qual o maior valor do tempo (dias) em que o anúncio está disponível
# na plataforma?
np.max(df1.loc[:,'availability_365'])

365

In [ ]:
# 23. Qual a data mais recente da última avaliação feita?
np.max(df1.loc[:,'last_review'])

'2019-07-08'

In [ ]:
# 24. Desenhe o histograma do tempo (dias) em que o anúncio está disponível
# na plataforma.
px.histogram(df1, 'availability_365', nbins=100)

In [ ]:
# 25. Desenhe o histograma da quantidade revisões mensais
px.histogram(df1, 'reviews_per_month', nbins=200)

In [ ]:
# 26. Desenhe o histograma da quantidade de anúncios do host do imóvel
px.histogram(df1.loc[df1['calculated_host_listings_count']<60], 'calculated_host_listings_count', nbins=200)

In [21]:
# 27. Desenhe um map da cidade de NY, localizando somente os top 10 imóveis
# com mais avaliações por região
data_plot = df1[['neighbourhood_group', 'number_of_reviews', 'latitude', 'longitude']].groupby('neighbourhood_group').head(10)
data_plot = data_plot.sort_values(['neighbourhood_group', 'number_of_reviews'], ascending=[True,False])

map = folium.Map(
    location=[data_plot['latitude'].mean(),
              data_plot['longitude'].mean()],
              control_scale=True
)

for index, location_info in data_plot.iterrows():
  folium.Marker([location_info['latitude'], location_info['longitude']],
                popup=location_info['number_of_reviews'],
                ).add_to(map)
map

In [22]:
# 28. Desenhe um mapa da cidade de NY, localizando somente os top 50 maiores
# preços por região
data_plot = df1[['neighbourhood_group', 'price', 'latitude', 'longitude']].groupby('neighbourhood_group').head(50)
data_plot = data_plot.sort_values(['neighbourhood_group', 'price'], ascending=[True,False])

map = folium.Map(
    location=[data_plot['latitude'].mean(),
              data_plot['longitude'].mean()],
              control_scale=True
)

for index, location_info in data_plot.iterrows():
  folium.Marker([location_info['latitude'], location_info['longitude']],
                popup=location_info['price'],
                ).add_to(map)
map

In [23]:
# 29. Desenhe um mapa da cidade de NY, localizando somente os top 10 menores
# preços por região
data_plot = df1[['neighbourhood_group', 'price', 'latitude', 'longitude']].groupby('neighbourhood_group').head(10)
data_plot = data_plot.sort_values(['neighbourhood_group', 'price'], ascending=[True,True])

map = folium.Map(
    location=[data_plot['latitude'].mean(),
              data_plot['longitude'].mean()],
              control_scale=True
)

for index, location_info in data_plot.iterrows():
  folium.Marker([location_info['latitude'], location_info['longitude']],
                popup=location_info['price'],
                ).add_to(map)
map

In [28]:
# 30. Desenhe um gráfico de barras, mostrando a média de aluguel diária de cada
# região
data_plot = df1[['neighbourhood_group', 'price']].groupby(['neighbourhood_group']).mean().reset_index()

px.bar(
   data_plot,
   x='neighbourhood_group',
   y='price'
)

In [29]:
# 31. Desenhe um gráfico de barras, mostrando a mediana de aluguel diária de cada
# região
data_plot = df1[['neighbourhood_group', 'price']].groupby(['neighbourhood_group']).median().reset_index()

px.bar(
   data_plot,
   x='neighbourhood_group',
   y='price'
)

In [30]:
# 32. Desenhe um gráfico de barras, mostrando a soma de todos os valores da diária
# de cada região
data_plot = df1[['neighbourhood_group', 'price']].groupby(['neighbourhood_group']).sum().reset_index()

px.bar(
   data_plot,
   x='neighbourhood_group',
   y='price'
)

In [59]:
# 33. Desenhe um gráfico de barras, mostrando o menor valor de diária de cada
# região
cols = ['neighbourhood_group', 'price']
data_plot = df1.loc[df1['price']>0,cols].groupby(['neighbourhood_group']).min().reset_index()

px.bar(
   data_plot,
   x='neighbourhood_group',
   y='price'
)

In [55]:
cols = ['neighbourhood_group', 'price']
data = df1.loc[df1['price']==0, cols]
data[df1['price']==0].value_counts()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



neighbourhood_group  price
Brooklyn             0        9
Bronx                0        1
dtype: int64